In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/gcc4_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/gcc4_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/gcc4_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/gcc4_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

Using TensorFlow backend.


(2156208, 2) (1724990, 2) (1773289, 2) (2224199, 2)
reset_index 완료


,bin,label
0,55,1
1,89,0
2,e5,0
3,53,0
4,83,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)

# 테스트데이터 (gcc 최적화버전 3 one hot encoding)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2156208, 257) (1724990, 257) (1773289, 257) (2224199, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2151901
1       4307
Name: label, dtype: int64 
 0    1721737
1       3253
Name: label, dtype: int64
0    1770091
1       3198
Name: label, dtype: int64 
 0    2221208
1       2991
Name: label, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    322630
ff    186095
8b    155808
45    113507
89    100993
       ...  
a9       269
9e       266
96       252
b1       252
a6       239
Name: bin, Length: 256, dtype: int64

In [6]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
# left_idx, right_idx = -10, 11 # 10개씩
left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 228103


In [7]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 168654


In [8]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 171069


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 162305


In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(228103, 256) (228103,)
(168654, 256) (168654,)
(171069, 256) (171069,)
(162305, 256) (162305,)


In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(228103, 1, 256) (228103, 1, 1)
(168654, 1, 256) (168654, 1, 1)
(171069, 1, 256) (171069, 1, 1)
(162305, 1, 256) (162305, 1, 1)


In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 012 / 3
x_gcc6_012_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32), axis=0)
y_gcc6_012_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32), axis=0)
print(x_gcc6_012_32.shape, y_gcc6_012_32.shape)
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

# 013 / 2
x_gcc6_013_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_3_32), axis=0)
y_gcc6_013_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_3_32), axis=0)
print(x_gcc6_013_32.shape, y_gcc6_013_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# 023 / 1
x_gcc6_023_32 = np.concatenate((x_gcc6_0_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_023_32 = np.concatenate((y_gcc6_0_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_023_32.shape, y_gcc6_023_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)

# 123 / 0
x_gcc6_123_32 = np.concatenate((x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_123_32 = np.concatenate((y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_123_32.shape, y_gcc6_123_32.shape)
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)

(567826, 1, 256) (567826, 1, 1)
(162305, 1, 256) (162305, 1, 1)
(559062, 1, 256) (559062, 1, 1)
(171069, 1, 256) (171069, 1, 1)
(561477, 1, 256) (561477, 1, 1)
(168654, 1, 256) (168654, 1, 1)
(502028, 1, 256) (502028, 1, 1)
(228103, 1, 256) (228103, 1, 1)


## Gcc6 Optimizer 012 학습 / 3 test

In [13]:
# (10) 양방향 LSTM 모델링 작업 - 012 / 3 
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed
import numpy as np

# batch_shape => None, 1, 256 뜻 행 batch 크기 안정하고, 1은 열이 1개, 256은 열안의 항목이 256개 
xInput = Input(batch_shape=(None, 1, 256)) # 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model1.summary())

print('======Training stage======')

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

model1.fit(x_gcc6_012_32, y_gcc6_012_32,
          batch_size = 32, #2,
          #validation_split=0.2,
          epochs=3,
          callbacks=[early_stopping])
print('학습완료')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 32)             34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________
None
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
567826/567826 [==============================] - 55s 96us/step - loss: 0.0474 - accuracy: 0.9885
Epoch 2/3
567826/567826 [==============================] - 53s 93us/st

학습완료


In [14]:
# (11) 테스트 Evaluate 결과 - 012 / 3 
# 정밀도가 의미하는 바는 => 내가 이진분류에서 실제로 잘 맞췄느냐

score, acc = model1.evaluate(x_gcc6_3_32, y_gcc6_3_32)
print('Test performance: accuracy={0}, loss={1}'.format(acc, score))

# 모델 predict
pr1 = model1.predict(x_gcc6_3_32)
print(pr1.shape) # pr1[:10]

162305/162305 [==============================] - 5s 29us/step
Test performance: accuracy=0.9881149530410767, loss=0.05218947108499145
(162305, 1, 1)


In [15]:
# (12) 테스트 평가지표 변수 작업 - 012 / 3

# 테스트 predcit 결과들
pred = np.round(np.array(pr1).flatten().tolist())

# 테스트 predict 정답들
y_test = np.array(y_gcc6_3_32).flatten().tolist()

print(len(pred), len(y_test)) # 두 개가 같으면 됨.
print('--- Gcc 6 이진분류 30gram 012 Train, 3 Test --- ')

# (13) 테스트 평가지표 변수 작업

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# score 출력
print('accuracy_score', float(accuracy_score(y_test, pred)))
print('recall_score', float(recall_score(y_test, pred)))
print('precision_score', float(precision_score(y_test, pred)))
print('f1_score', f1_score(y_test, pred))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print('\nconfusion_matrix\n', confusion_matrix(y_test, pred))

# ROC Curve
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test, pred)
print('roc_curve 면적', roc_auc)

162305 162305
--- Gcc 6 이진분류 30gram 012 Train, 3 Test --- 
accuracy_score 0.9881149687317088
recall_score 0.3725752508361204
precision_score 0.9545844044558698
f1_score 0.5359634351695934

confusion_matrix
 [[159262     53]
 [  1876   1114]]
roc_curve 면적 0.6861212882872189


## Gcc6 Optimizer 013 학습 / 2 test

In [16]:
# (10) 양방향 LSTM 모델링 작업 - # 013 / 2
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed
import numpy as np

# batch_shape => None, 1, 256 뜻 행 batch 크기 안정하고, 1은 열이 1개, 256은 열안의 항목이 256개 
xInput = Input(batch_shape=(None, 1, 256)) # 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model2 = Model(xInput, xOutput)
model2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model2.summary())

print('======Training stage======')

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

model2.fit(x_gcc6_013_32, y_gcc6_013_32,
          batch_size = 32, #2,
          #validation_split=0.2,
          epochs=3,
          callbacks=[early_stopping])
print('학습완료')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 256)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1, 32)             34944     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________
None
======Training stage======
Epoch 1/3
559062/559062 [==============================] - 58s 104us/step - loss: 0.0468 - accuracy: 0.9889
Epoch 2/3
559062/559062 [==============================] - 61s 110us/step - loss: 0.0408 - accuracy: 0.9894
Epoch 3/3
559062/559062 [==============================] - 55s 99us/step - loss: 0.0411 - accuracy: 0.9894


학습완료


In [17]:
# (11) 테스트 Evaluate 결과 - 013 / 2 
# 정밀도가 의미하는 바는 => 내가 이진분류에서 실제로 잘 맞췄느냐

score2, acc2 = model2.evaluate(x_gcc6_2_32, y_gcc6_2_32)
print('Test performance: accuracy={0}, loss={1}'.format(acc2, score2))

# 모델 predict
pr2 = model2.predict(x_gcc6_2_32)
print(pr2.shape) # pr2[:10]

171069/171069 [==============================] - 9s 50us/step
Test performance: accuracy=0.986677885055542, loss=0.05815501352666119
(171069, 1, 1)


In [18]:
# (12) 테스트 평가지표 변수 작업 - 013 / 2

# 테스트 predcit 결과들
pred2 = np.round(np.array(pr2).flatten().tolist())

# 테스트 predict 정답들
y_test2 = np.array(y_gcc6_2_32).flatten().tolist()

print(len(pred2), len(y_test2)) # 두 개가 같으면 됨.
print('--- Gcc 6 이진분류 30gram 013 Train, 2 Test --- ')

# (13) 테스트 평가지표 변수 작업

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# score 출력
print('accuracy_score', float(accuracy_score(y_test2, pred2)))
print('recall_score', float(recall_score(y_test2, pred2)))
print('precision_score', float(precision_score(y_test2, pred2)))
print('f1_score', f1_score(y_test2, pred2))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print('\nconfusion_matrix\n', confusion_matrix(y_test2, pred2))

# ROC Curve
from sklearn.metrics import roc_auc_score
roc_auc2 = roc_auc_score(y_test2, pred2)
print('roc_curve 면적', roc_auc2)

171069 171069
--- Gcc 6 이진분류 30gram 013 Train, 2 Test --- 
accuracy_score 0.9866778902080447
recall_score 0.29965592743196745
precision_score 0.9599198396793587
f1_score 0.4567342073897496

confusion_matrix
 [[167832     40]
 [  2239    958]]
roc_curve 면적 0.6497088253248285


## Gcc6 Optimizer 023 학습 / 1 test

In [19]:
# (10) 양방향 LSTM 모델링 작업 - # 023 / 1
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed
import numpy as np

# batch_shape => None, 1, 256 뜻 행 batch 크기 안정하고, 1은 열이 1개, 256은 열안의 항목이 256개 
xInput = Input(batch_shape=(None, 1, 256)) # 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model3 = Model(xInput, xOutput)
model3.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model3.summary())

print('======Training stage======')

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

model3.fit(x_gcc6_023_32, y_gcc6_023_32,
          batch_size = 32, #2,
          #validation_split=0.2,
          epochs=3,
          callbacks=[early_stopping])
print('학습완료')

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1, 256)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1, 32)             34944     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 1, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________
None
======Training stage======
Epoch 1/3
561477/561477 [==============================] - 67s 120us/step - loss: 0.0466 - accuracy: 0.9892
Epoch 2/3
561477/561477 [==============================] - 64s 113us/step - loss: 0.0408 - accuracy: 0.9896
Epoch 3/3
561477/561477 [==============================] - 52s 93us/step - loss: 0.0406 - accuracy: 0.9896


학습완료


In [20]:
# (11) 테스트 Evaluate 결과 - 023 / 1 
# 정밀도가 의미하는 바는 => 내가 이진분류에서 실제로 잘 맞췄느냐

score3, acc3 = model3.evaluate(x_gcc6_1_32, y_gcc6_1_32)
print('Test performance: accuracy={0}, loss={1}'.format(acc3, score3))

# 모델 predict
pr3 = model3.predict(x_gcc6_1_32)
print(pr3.shape) # pr3[:10]

168654/168654 [==============================] - 5s 27us/step
Test performance: accuracy=0.9858408570289612, loss=0.05623170098015692
(168654, 1, 1)


In [21]:
# (12) 테스트 평가지표 변수 작업 - 023 / 1

# 테스트 predcit 결과들
pred3 = np.round(np.array(pr3).flatten().tolist())

# 테스트 predict 정답들
y_test3 = np.array(y_gcc6_1_32).flatten().tolist()

print(len(pred3), len(y_test3)) # 두 개가 같으면 됨.
print('--- Gcc 6 이진분류 30gram 023 Train, 1 Test --- ')

# (13) 테스트 평가지표 변수 작업

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# score 출력
print('accuracy_score', float(accuracy_score(y_test3, pred3)))
print('recall_score', float(recall_score(y_test3, pred3)))
print('precision_score', float(precision_score(y_test3, pred3)))
print('f1_score', f1_score(y_test3, pred3))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print('\nconfusion_matrix\n', confusion_matrix(y_test3, pred3))

# ROC Curve
from sklearn.metrics import roc_auc_score
roc_auc3 = roc_auc_score(y_test3, pred3)
print('roc_curve 면적', roc_auc3)

168654 168654
--- Gcc 6 이진분류 30gram 023 Train, 1 Test --- 
accuracy_score 0.9858408338966167
recall_score 0.27982779827798276
precision_score 0.9518828451882845
f1_score 0.432509505703422

confusion_matrix
 [[165356     46]
 [  2342    910]]
roc_curve 면적 0.6397748439885096


## Gcc6 Optimizer 123 학습 / 0 test

In [22]:
# (10) 양방향 LSTM 모델링 작업 - # 123 / 0
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed
import numpy as np

# batch_shape => None, 1, 256 뜻 행 batch 크기 안정하고, 1은 열이 1개, 256은 열안의 항목이 256개 
xInput = Input(batch_shape=(None, 1, 256)) # 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model4 = Model(xInput, xOutput)
model4.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model4.summary())

print('======Training stage======')

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

model4.fit(x_gcc6_123_32, y_gcc6_123_32,
          batch_size = 32, #2,
          #validation_split=0.2,
          epochs=3,
          callbacks=[early_stopping])
print('학습완료')

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1, 256)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1, 32)             34944     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 1, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________
None
======Training stage======
Epoch 1/3
502028/502028 [==============================] - 44s 87us/step - loss: 0.0554 - accuracy: 0.9865
Epoch 2/3
502028/502028 [==============================] - 53s 106us/step - loss: 0.0489 - accuracy: 0.9869
Epoch 3/3
502028/502028 [==============================] - 52s 103us/step - loss: 0.0488 - accuracy: 0.9869


학습완료


In [23]:
# (11) 테스트 Evaluate 결과 - 123 / 0 
# 정밀도가 의미하는 바는 => 내가 이진분류에서 실제로 잘 맞췄느냐

score4, acc4 = model4.evaluate(x_gcc6_0_32, y_gcc6_0_32)
print('Test performance: accuracy={0}, loss={1}'.format(acc4, score4))

# 모델 predict
pr4 = model4.predict(x_gcc6_0_32)
print(pr4.shape) # pr4[:10]

228103/228103 [==============================] - 9s 38us/step
Test performance: accuracy=0.9928234219551086, loss=0.04770080600238725
(228103, 1, 1)


In [24]:
# (12) 테스트 평가지표 변수 작업 - 123 / 0

# 테스트 predcit 결과들
pred4 = np.round(np.array(pr4).flatten().tolist())

# 테스트 predict 정답들
y_test4 = np.array(y_gcc6_0_32).flatten().tolist()

print(len(pred4), len(y_test4)) # 두 개가 같으면 됨.
print('--- Gcc 6 이진분류 30gram 123 Train, 0 Test --- ')

# (13) 테스트 평가지표 변수 작업

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# score 출력
print('accuracy_score', float(accuracy_score(y_test4, pred4)))
print('recall_score', float(recall_score(y_test4, pred4)))
print('precision_score', float(precision_score(y_test4, pred4)))
print('f1_score', f1_score(y_test4, pred4))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print('\nconfusion_matrix\n', confusion_matrix(y_test4, pred4))

# ROC Curve
from sklearn.metrics import roc_auc_score
roc_auc4 = roc_auc_score(y_test4, pred4)
print('roc_curve 면적', roc_auc4)

228103 228103
--- Gcc 6 이진분류 30gram 123 Train, 0 Test --- 
accuracy_score 0.992823417491221
recall_score 1.0
precision_score 0.7245498906276291
f1_score 0.840277100204898

confusion_matrix
 [[222160   1637]
 [     0   4306]]
roc_curve 면적 0.9963426676854471


## 기타

In [25]:
'''
# gcc7_0~3
gcc7_0_32 = pd.read_csv("data_final/gcc7_0_32.csv")
gcc7_1_32 = pd.read_csv("data_final/gcc7_1_32.csv")
gcc7_2_32 = pd.read_csv("data_final/gcc7_2_32.csv")
gcc7_3_32 = pd.read_csv("data_final/gcc7_3_32.csv")

# gcc8_0~3
gcc8_0_32 = pd.read_csv("data_final/gcc8_0_32.csv")
gcc8_1_32 = pd.read_csv("data_final/gcc8_1_32.csv")
gcc8_2_32 = pd.read_csv("data_final/gcc8_2_32.csv")
gcc8_3_32 = pd.read_csv("data_final/gcc8_3_32.csv")

# gcc9_0~3
gcc9_0_32 = pd.read_csv("data_final/gcc9_0_32.csv")
gcc9_1_32 = pd.read_csv("data_final/gcc9_1_32.csv")
gcc9_2_32 = pd.read_csv("data_final/gcc9_2_32.csv")
gcc9_3_32 = pd.read_csv("data_final/gcc9_3_32.csv")
'''

'\n# gcc7_0~3\ngcc7_0_32 = pd.read_csv("data_final/gcc7_0_32.csv")\ngcc7_1_32 = pd.read_csv("data_final/gcc7_1_32.csv")\ngcc7_2_32 = pd.read_csv("data_final/gcc7_2_32.csv")\ngcc7_3_32 = pd.read_csv("data_final/gcc7_3_32.csv")\n\n# gcc8_0~3\ngcc8_0_32 = pd.read_csv("data_final/gcc8_0_32.csv")\ngcc8_1_32 = pd.read_csv("data_final/gcc8_1_32.csv")\ngcc8_2_32 = pd.read_csv("data_final/gcc8_2_32.csv")\ngcc8_3_32 = pd.read_csv("data_final/gcc8_3_32.csv")\n\n# gcc9_0~3\ngcc9_0_32 = pd.read_csv("data_final/gcc9_0_32.csv")\ngcc9_1_32 = pd.read_csv("data_final/gcc9_1_32.csv")\ngcc9_2_32 = pd.read_csv("data_final/gcc9_2_32.csv")\ngcc9_3_32 = pd.read_csv("data_final/gcc9_3_32.csv")\n'

In [26]:
'''
# (번외) prediction 결과 확인
pr_ls = np.array(pr1).flatten().tolist()

count=0
for num, i in enumerate(pr_ls):
    if i >= 0.5:
        count+=1
#        print(i)
print(count)
'''

'\n# (번외) prediction 결과 확인\npr_ls = np.array(pr1).flatten().tolist()\n\ncount=0\nfor num, i in enumerate(pr_ls):\n    if i >= 0.5:\n        count+=1\n#        print(i)\nprint(count)\n'